Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Отлично, что правильно использованы train/valid/test наборы.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Я оставил несколько советов, надеюсь, что они будут учтены в будущих проектах.
* Тебе удалось добиться очень хорошего качества, поздравляю!
* Данный проект зачтен. Удачи в дальнейшем обучении!

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from tqdm.notebook import tqdm

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка и первичный анализ данных проведены успешно.
</div>

В данных нет пропущенных строк. Значения в столбцах выглядят корректно предобработка данных не тебуется. Прежде чем перейти к разбиению данных на выбоки проведем замену тиов данных.

In [5]:
df.dtypes.apply(lambda x: x.name).to_dict()

{'calls': 'float64',
 'minutes': 'float64',
 'messages': 'float64',
 'mb_used': 'float64',
 'is_ultra': 'int64'}

In [6]:
# заменим типы в таблице по словарю df_type_dict
df_type_dict = {
 'calls': 'int32',
 'minutes': 'float32',
 'messages': 'int16',
 'mb_used': 'float32',
 'is_ultra': 'int8'}

df = df.astype(df_type_dict)

# убедимся что замена прошла так как задумано
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int32
minutes     3214 non-null float32
messages    3214 non-null int16
mb_used     3214 non-null float32
is_ultra    3214 non-null int8
dtypes: float32(2), int16(1), int32(1), int8(1)
memory usage: 47.2 KB


## 2. Разбейте данные на выборки

Имена столбцов с "фичами" сохраним в списке x_col, столбец который мы будем предсказывать в списке y_col

In [7]:
x_col = df.columns.drop('is_ultra')
x_col

Index(['calls', 'minutes', 'messages', 'mb_used'], dtype='object')

In [8]:
y_col = ['is_ultra']

Разобьем данные сначала на тестовую и полную тренировачную выбоки, а полную тренировачную выбоку разобъем на неоседственно трениовачную выбоку и валидационную выбоку

In [9]:
df_train_full, df_test = train_test_split(df, test_size=0.25, random_state=42)
df_train, df_valid = train_test_split(df_train_full, test_size=0.25, random_state=42)

<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно было напечатать размеры полученных наборов. Иногда это помогает избежать ошибок.
</div>

## 3. Исследуйте модели

Исследуем следующие модели классификации:
* дерево решений
* случайный лес
* логистическая регрессия
* CatBoost

### Дерево решений
Для дерева решений будем менять глубину дерева и найдем такую глубину дерева которая дает наилучшее accuracy на валидационной выборке

In [10]:
best_tree_model = None
best_tree_depth = 0
best_tree_score = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    model.fit(df_train[x_col], df_train[y_col].values.ravel())
    score = accuracy_score(df_valid[y_col], model.predict(df_valid[x_col]))
    if score > best_tree_score:
        best_tree_model = model
        best_tree_depth = depth
        best_tree_score = score
print('Доля првильных ответов на валидационной выбоке:', best_tree_score, 'для дерева глубиной', best_tree_depth)

Доля првильных ответов на валидационной выбоке: 0.7860696517412935 для дерева глубиной 6


### Случайный лес
Для случайного леса так же будем переберать глубину дерева и кроме того число деревьев.

In [11]:
best_forest_model = None
best_forest_depth = 0
best_forest_n_estimators = 0
best_forest_score = 0
for n_estimators in tqdm(range (1, 51)):
    for depth in range(1, 11):
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=depth, random_state=42)
        model.fit(df_train[x_col], df_train[y_col].values.ravel())
        score = accuracy_score(df_valid[y_col], model.predict(df_valid[x_col]))
        if score > best_forest_score:
            best_forest_model = model
            best_forest_depth = depth
            best_forest_n_estimators = n_estimators
            best_forest_score = score

print('Доля првильных ответов на валидационной выбоке:', best_forest_score, 'для деревьев глубиной', best_forest_depth)
print('Число деревьев:', best_forest_n_estimators)


Доля првильных ответов на валидационной выбоке: 0.8109452736318408 для деревьев глубиной 7
Число деревьев: 3


### Логистическая регрессия
Для логистической регрессии параметры подберем ручным перебором

In [12]:
logistic_regression_model = LogisticRegression(random_state=42, solver='liblinear', penalty='l1', intercept_scaling=10)
logistic_regression_model.fit(df_train[x_col], df_train[y_col].values.ravel())
logistic_regression_score = accuracy_score(df_valid[y_col], logistic_regression_model.predict(df_valid[x_col]))

print('Доля првильных ответов на валидационной выбоке:', logistic_regression_score)

Доля првильных ответов на валидационной выбоке: 0.7545605306799337


<div class="alert alert-block alert-info">
<b>Совет: </b> Не имеет смысла печатать так много знаков после запятой, достаточно 2-3: "0.XXX".
</div>

### CatBoost
CatBoost используем с параметрами в основном по умолчанию, ограничим лишь число итераций (200) и количетво выводимой информации (информация выводится каждые 50 итеаций).

In [13]:
cat_boost_model = CatBoostClassifier(verbose=50, early_stopping_rounds=200, random_state=42)
cat_boost_model.fit(df_train[x_col], df_train[y_col].values.ravel(),
                    eval_set=(df_valid[x_col], df_valid[y_col].values.ravel()))

cat_boost_score = accuracy_score(df_valid[y_col], cat_boost_model.predict(df_valid[x_col]))

print('Доля првильных ответов на валидационной выбоке:', cat_boost_score)

Learning rate set to 0.083029
0:	learn: 0.6599045	test: 0.6590437	best: 0.6590437 (0)	total: 51ms	remaining: 50.9s
50:	learn: 0.4075682	test: 0.4569395	best: 0.4563372 (46)	total: 757ms	remaining: 14.1s
100:	learn: 0.3599120	test: 0.4584878	best: 0.4549443 (59)	total: 1.62s	remaining: 14.4s
150:	learn: 0.3128304	test: 0.4709159	best: 0.4549443 (59)	total: 2.41s	remaining: 13.5s
200:	learn: 0.2735947	test: 0.4765392	best: 0.4549443 (59)	total: 3.27s	remaining: 13s
250:	learn: 0.2422955	test: 0.4878269	best: 0.4549443 (59)	total: 4.05s	remaining: 12.1s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4549442983
bestIteration = 59

Shrink model to first 60 iterations.
Доля првильных ответов на валидационной выбоке: 0.8009950248756219


<div class="alert alert-block alert-success">
<b>Успех:</b> Подбор параметров реализован абсолютно верно!
</div>

### Итоговая модель
Финальную модель построим комбинацией трех моделей: модели случайного леса, модели логистической регрессии и CatBoost. Решающее правило при этом определим следующим образом: если две или три модели советуют тариф "Ультра" (прогнозируют единицу), то пользователю предлагается тариф "Ультра" (итоговый прогноз единица), иначе "Смарт" (нуль). Введем функцию sum_score_calc которая оценивает accuracy по данному решающему правилу и оценим accuracy на валидационной выборке

In [14]:
def sum_score_calc(df):
    predict_sum = (best_forest_model.predict(df[x_col]) + 
                   logistic_regression_model.predict(df[x_col]) +
                   cat_boost_model.predict(df[x_col]))
    score = accuracy_score(df[y_col], (predict_sum >= 2)*1)
    return score

In [15]:
print('Доля првильных ответов на валидационной выбоке:', sum_score_calc(df_valid))

Доля првильных ответов на валидационной выбоке: 0.8126036484245439


<div class="alert alert-block alert-success">
<b>Успех:</b> Интересная идея объеденить несколько моделей. Вот <a href="https://habr.com/ru/post/430280/">статья</a> на эту тему.
</div>

Как видно введение решающего правило дало небольшой прирост accuracy на валидационной выборке по сравнению с моделью случайного леса (81,26 % против 81,09 % у леса)

## 4. Проверьте модель на тестовой выборке

In [16]:
print('Доля првильных ответов на тестовой выбоке:', sum_score_calc(df_test))

Доля првильных ответов на тестовой выбоке: 0.8171641791044776


На тестовой выборке итоговая модель дает accuracy близкий к тому что было получено на валидации, что может говорить о стабильности работы алгоритма.

<div class="alert alert-block alert-success">
<b>Успех:</b> Тестирование сделано корректно.
</div>

## 5. (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность, сравним accuracy итоговой модели на тестовых данных с accuracy модели которая всегда предсказывает тариф "Смарт" (нуль).

In [17]:
accuracy_score(df_test[y_col], [0]*len(df_test))

0.7027363184079602

Модель которая всегда предсказывает тариф "Смарт" на тестовой выборке дает accuracy - 70,27%. Учитывая что итоговая модель на тестовой выборке дает accuracy - 81,7%, эту модель можно считать адекватной.

<div class="alert alert-block alert-success">
<b>Успех:</b> Идея и реализация корректны. В будущем советую использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html"> DummyClassifier </a>. 
</div>

## Выводы

1. В работе была оценена возможность использования следующих алгоритмов:
    * дерево решений
    * случайный лес
    * логистическая регрессия
    * CatBoost
    
* Оптимизированы параметры алгоритмов
* Предложено решающее правило по которому сформирована итоговая модель
* Проведена оценка accuracy алгоритмов
* Итоговая модель имеет accuracy на тестовой выборке 81,7 % и может быть использована для построения системы, способной на основе поведения клиентов предложить им новый тариф: «Смарт» или «Ультра».
    



<div class="alert alert-block alert-success">
<b>Успех:</b> Радует, что ты не забыл про общий вывод.
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
